In [2]:
using Images
using PyPlot
using Statistics
using LsqFit
using DelimitedFiles

#setting the correct GUI backend for proper figure display
include("utils.jl")
set_gui();

In [3]:
#provide a path to the kymograph
img_path = "//example/"
img_name = "test.tif"
loadpath = string(img_path,img_name);

timestep = 2;  #dt between kymograph lines in seconds
mag = 0.129;   #pixel size in micrometers

main_channel = 1; #primary channel to trace
secondary_channel = 3; #secondary channel 

#load the full tiff and display the channel we will be analyzing
img = Float32.(load(loadpath));
img_a = Float32.(img);
kymograph1 = copy(img_a[:,:,main_channel]);
kymograph2 = copy(img_a[:,:,secondary_channel]);

f1 = figure();
subplot(121, title="main channel");
imshow(kymograph1);

subplot(122, title="secondary channel");
imshow(kymograph2);

In [4]:
#manually select a portion of the kymograph containing only one moving spot
#click twice on the kymograph: once on top left corner of our future crop, once on bottom right corner
coords = zeros(0);
coords = ginput_routine(2,kymograph1);

#display the cropped kymograph defined by our clicks
figure();
crop = kymograph1[coords[1]:coords[2],coords[3]:coords[4]];
imshow(crop);

In [5]:
#record background (BG) intensity
#click twice on the cropped kymograph selecting a rectangle without moving particles
BG_coords = ginput_routine(2,crop);

In [6]:
#record initial intensity and initial position of the moving particle
#click twice on the cropped kymograph selecting a rectangle containing >=10 initial positions of the moving particle
peak_coords = ginput_routine(2,crop);

In [7]:
crop2 = kymograph2[coords[1]:coords[2],coords[3]:coords[4]];
f_ch2=figure();
imshow(crop2);
BG2_coords = ginput_routine(2,crop);

In [8]:
#prepare initial values for the spot fitting function 
d = mean(crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]);                #mean background intensity
dsd = stdm((crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]), d);         #background SD
a = maximum(crop[peak_coords[1]:peak_coords[2],peak_coords[3]:peak_coords[4]]) - d; #peak height
b = (peak_coords[3] + peak_coords[4])/2;                                            #peak position
c = - 2;   width_tol = 0.2;                                                         #peak width and its tolerance
t = size(crop,1);                                                                   #number of timepoints (kymograph lines)
d2 = mean(crop2[BG2_coords[1]:BG2_coords[2],BG2_coords[3]:BG2_coords[4]]);          #channel 2 mean background intensity
dsd2 = stdm((crop2[BG2_coords[1]:BG2_coords[2],BG2_coords[3]:BG2_coords[4]]), d2);  #channel 2 background SD 

#preallocating fitting result arrays for speed
timesec = zeros(t);
timepix = zeros(t);
pixel = zeros(t);
position = zeros(t);
intensity = zeros(t);
intensity2 = zeros(t);
position2 = zeros(t);
peakwidth = zeros(t);
background = zeros(t);

In [9]:
#we fit each line in our cropped kymograph with a gaussian peak of height a and width c at position b
#after fitting spot position in channel 1, we will measure intensity in channel 2 at these positions
let a = a, b = b, c = c, d = d, dsd = dsd, t = t, width_tol = width_tol, a2 = crop2[Int.(round(b))]
    for i in 1:t
        ydata = crop[i,:];
        xdata = collect(1:length(ydata));
        
        (a, b, c, d) = fit_gaussian(xdata, ydata, a, b, c, d, dsd)

        
        timepix[i] = i;
        pixel[i] = b;
        timesec[i] = i*timestep;
        position[i] = b*mag;
        peakwidth[i] = -c;
        background[i] = d;
        
        if i > 1
            if peakwidth[i] > peakwidth[1]*(1+width_tol)
                peakwidth[i]=peakwidth[1]*(1+width_tol);
            elseif peakwidth[i] < peakwidth[1]*(1-width_tol)
                peakwidth[i]=peakwidth[1]*(1-width_tol);
            end
        end
        
        intensity[i] = a*peakwidth[i];
        
        if Int.(round(pixel[i]))-1 < 2
            b2 = 2;
        elseif Int.(round(pixel[i]))+1 > size(crop2,2)
            b2 = size(crop2,2) - 1;
        else
            b2 = Int.(round(pixel[i]));
        end
        a2 = maximum(crop2[i,b2-1:b2+1]) - d2;
        intensity2[i] = a2*peakwidth[i];
         
        IJulia.clear_output(true);
        println(string("Processing line ", i, " of ", t, "; a = ", round(a), "; b = ", round(b)))
    end
end

Processing line 350 of 479; a = 748.0; b = 9.0


LoadError: LinearAlgebra.SingularException(4)

In [10]:
#plot fitted coordinates
fig = figure(img_name);
PyPlot.title(img_name);
subplot(141, title="channel 1", xlabel="position", ylabel="time");
imshow(crop);

subplot(142, title="fitted channel 1", xlabel="position", ylabel="time");
imshow(crop);
ylim(t,0);
xlim(0,length(crop[1,:]));
grid("on");
pl = scatter(pixel, timepix, facecolor="none", edgecolors="red", s = 30);

subplot(143, title="channel 2", xlabel="position", ylabel="time");
imshow(crop2);

subplot(144, title="fitted channel 2", xlabel="position", ylabel="time");
imshow(crop2);
ylim(t,0);
xlim(0,length(crop[1,:]));
grid("on");
pl = scatter(pixel, timepix, facecolor="none", edgecolors="red", s = 30);


In [11]:
fig2 = figure(figsize=(8,5));
xlabel("time (s)");
font1 = Dict("color"=>"blue");
ylabel("position (micrometer)",fontdict=font1);
p2 = plot(timesec,position, color="blue");
xlim(0, maximum(timesec));
ylim(0, maximum(position));
ax = gca()

fig2.subplots_adjust(right=0.85);
ax2 = ax.twinx();
font2 = Dict("color"=>"orange");
ylabel("intensity (a.u.)",fontdict=font2);
p3 = plot(timesec,intensity, color="orange");
ylim(0, maximum(intensity));
p4 = plot(timesec,intensity2, color="green");

PyPlot.title(img_name);

In [12]:
let savepath = string(img_path, "traced/")
    if isdir(savepath) == false
        mkdir(savepath)
    end 
end

In [13]:
function fcheck()
    filecount = 1;
    savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    while isfile(savename) == true
        filecount = filecount + 1
        savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    end
    return filecount
end

filecount = fcheck();

savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt")
open(savename, "w") do io;
    writedlm(io, [timesec position intensity intensity2]);
end

subcropname1 = string(img_path, "traced/", img_name[1:end-4], "-", filecount, "_ch1_subcrop.tif");
save(subcropname1, colorview(Gray, crop));

subcropname2 = string(img_path, "traced/", img_name[1:end-4], "-", filecount, "_ch2_subcrop.tif");
save(subcropname2, colorview(Gray, crop2));

In [15]:
#optional, checking the saved files
fig_check = figure(img_name);
PyPlot.title(img_name);
subplot(121, title="channel 1 saved", xlabel="position", ylabel="time");
imshow(Float32.(load(subcropname1)));

subplot(122, title="channel 2 saved", xlabel="position", ylabel="time");
imshow(Float32.(load(subcropname2)));

l = readdlm(savename, '\t', Float64);

fig_check2 = figure(figsize=(8,5));
xlabel("time (s)");
font1 = Dict("color"=>"blue");
ylabel("position (micrometer)",fontdict=font1);
p2 = plot(l[:,1],l[:,2], color="blue");
xlim(0, maximum(timesec));
ylim(0, maximum(position));
ax = gca()

fig_check2.subplots_adjust(right=0.85);
ax2 = ax.twinx();
font2 = Dict("color"=>"orange");
ylabel("intensity (a.u.)",fontdict=font2);
p3 = plot(l[:,1],l[:,3], color="orange");
ylim(0, maximum(intensity));
p4 = plot(l[:,1],l[:,4], color="green");

PyPlot.title(img_name);